In [ ]:
!pip install transformers==4.45.1 accelerate>=0.26.1 bitsandbytes transformers[torch] wandb tqdm datasets peft  langchain langchain_community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.1.1 requires accelerate<0.22.0,>=0.21.0, but you have accelerate 0.29.3 which is incompatible.
autogluon-multimodal 1.1.1 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-multimodal 1.1.1 requires scipy<1.13,>=1.5.4, but you have scipy 1.14.1 which is incompatible.
autogluon-multimodal 1.1.1 requires transformers[sentencepiece]<4.41.0,>=4.38.0, but you have transformers 4.45.1 which is incompatible.
autogluon-timeseries 1.1.1 requires

In [ ]:
!pip install s3fs==2023.9.2
!pip install boto3

  Using cached botocore-1.31.17-py3-none-any.whl.metadata (5.9 kB)
Using cached botocore-1.31.17-py3-none-any.whl (11.1 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.162
    Uninstalling botocore-1.34.162:
      Successfully uninstalled botocore-1.34.162
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amzn-sagemaker-studio-dataengineering-extensions 1.0.0 requires pytest, which is not installed.
amzn-sagemaker-studio-dataengineering-sessions 1.0 requires pytest, which is not installed.
amazon-sagemaker-sql-editor 0.1.14 requires aiobotocore<3,>=2.7.0, but you have aiobotocore 2.5.4 which is incompatible.
amazon-sagemaker-sql-editor 0.1.14 requires botocore<2,>=1.31.64, but you have botocore 1.31.17 which is incompatible.
boto3 1.34.162 requires botocore<1.35.0,>=1.34.162, but you have botocore 1.31.17 which is incompatible.

Documentation: [LORA](https://huggingface.co/docs/peft/en/package_reference/lora)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import wandb
from tqdm import tqdm
import os
from datasets import load_dataset
from getpass import getpass

# 1. Load the Dataset.

Loading the main split of the gsm8k dataset using the datasets module.

In [ ]:
import s3fs
from datasets import Dataset
import pandas as pd
import numpy as np
# Assuming df is your Pandas

fs=s3fs.S3FileSystem(anon=False) # initialize the file system

s3_url='s3://amazon-sagemaker-058264306111-us-east-1-e23504aef6c5/dzd_5l5kah6gnsnq3r/bzbm82rtfbxpgn/dev/IMDB Dataset.csv' # location of the file in s3 bucket

data=pd.read_csv(s3_url)

dataset = Dataset.from_pandas(data)

dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})

Structure of the dataset.

In [ ]:
dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})

Loading the train split of the dataset.

selecting the first 5000 instances of the dataset.

In [ ]:
dataset=dataset.select(range(5000))
dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 5000
})

# 2. Transforming the Dataset.
The code cell creates a function `_add_text` to format math questions and answers into a structured text format. It uses templates to create a prompt and appends the corresponding response, raising errors if either the question or response is missing.

In [ ]:
prompt_template = """Below is an Review. Predict the sentiment of the review in one word postive or negative. Question: {question}\n sentiment:"""
answer_template = """{response}"""
def _add_text(rec):
    question = rec["review"]
    response = rec["sentiment"]
    # check if both exists, else raise error
    if not question:
        raise ValueError(f"Expected an instruction in: {rec}")
    if not response:
        raise ValueError(f"Expected a response in: {rec}")
    rec["prompt"] = prompt_template.format(question=question)
    rec["answer"] = answer_template.format(response=response)
    rec["text"] = rec["prompt"] + rec["answer"]
    return rec


In [ ]:
train_set = dataset.map(_add_text)
train_set

  0%|          | 0/50000 [00:00<?, ?ex/s]

Dataset({
    features: ['review', 'sentiment', 'prompt', 'answer', 'text'],
    num_rows: 50000
})

### 3. Loading the  Model.

Create an account in [Hugging Face](https://huggingface.co/).

Go to [Access Tokens](https://huggingface.co/settings/tokens), and create a new token.

now search for the model with id `amd/AMD-OLMo-1B-SFT`, and check if you have access to the model. If you don't see any message saying this is restricted, then you are good to go. Else you will be asked if you want to get the consent to use this gated model, just fill out the details and you will have the access.

In [ ]:
model_id='amd/AMD-OLMo-1B-SFT'
os.environ['HF_TOKEN']=getpass("Enter HF Token : ")
model=AutoModelForCausalLM.from_pretrained(model_id,token=os.environ['HF_TOKEN'])
tokenizer=AutoTokenizer.from_pretrained(model_id)

Enter HF Token :  ········


In [ ]:
tokenizer.pad_token=tokenizer.eos_token
tokenizer.pad_token_id=tokenizer.eos_token_id


## Creating a sequential pipeline of model and tokenizer with parameters



`Model and Tokenizer`: Uses the specified model and tokenizer.

`Task: text-generation`, generating new text based on an input prompt.
Generation Settings:

`Maximum number of tokens` in the output: `512`.

Sampling `temperature: 0.3` (low value for more deterministic outputs).

`Top-p` sampling: `1` (includes all tokens for generation).

Enables sampling with `do_sample=True`.

`Repetition penalty`: `1.1` (reduces repetitive text).

The pipeline ensures the generated text includes the full context from the input prompt.

In [ ]:
import transformers

pipeline=transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=512,
    temperature=0.3,
    top_p=1,
    do_sample=True,
    repetition_penalty=1.1
)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
from langchain import HuggingFacePipeline
pretrained_model= HuggingFacePipeline(pipeline=pipeline)

# 4. Processing the Dataset.

`_preprocess_batch` function tokenizes text for training, truncating or padding inputs to a maximum length of 256 tokens and creating labels as a copy of the input IDs.

In [ ]:
from functools import partial
import copy
from typing import Dict, List
from transformers import DataCollatorForSeq2Seq

MAX_LENGTH = 256
def _preprocess_batch(batch: Dict[str, List]):
    model_inputs = tokenizer(batch["text"], max_length=MAX_LENGTH, truncation=True, padding='max_length')
    model_inputs["labels"] = copy.deepcopy(model_inputs['input_ids'])
    return model_inputs

In [ ]:
encoded_dataset=train_set.map(_preprocess_batch,batched=True,remove_columns=["review", "prompt", "sentiment"])

  0%|          | 0/50 [00:00<?, ?ba/s]

In [ ]:
processed_dataset = encoded_dataset.filter(lambda rec: len(rec["input_ids"]) <= MAX_LENGTH)

  0%|          | 0/50 [00:00<?, ?ba/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f4cb64947d0>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f4cb64947d0>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
data=processed_dataset.select(range(1000))

In [ ]:
data

Dataset({
    features: ['answer', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

## Creating a train-test split.

In [ ]:
split_dataset = data.train_test_split(test_size=0.2, seed=0)
print(split_dataset)

# takes a list of samples from a Dataset and collate them into a batch, as a dictionary of PyTorch tensors.
data_collator = DataCollatorForSeq2Seq(
        model = model, tokenizer=tokenizer, max_length=MAX_LENGTH, pad_to_multiple_of=8, padding='max_length')

DatasetDict({
    train: Dataset({
        features: ['answer', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['answer', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})


## Architecture of the model.

In [ ]:
model

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
  )
  (

#5. LORA config

**r** -> rank

Very low values like 8 or 4 are basically just telling the model a style of how you want the output to be formatted to but not care about the details as there isn’t enough parameters to distinguish much more.

 if you are trying to teach your model a bit of new stuff or concepts and new associations beyond what it already knows, you need to use higher rank.

**alpha** -> rank is scaling weights at 1.0

LORA weights will be then merged with the main weights of model at x 1.0


**target_modules** -> the layers we are targeting.

in the output of the above codecell, we can see different layers, so we were targetting the query module, key module and output module in the self_attention.

In [ ]:
from peft import LoraConfig, get_peft_model,prepare_model_for_kbit_training

LORA_R = 256 # 512
LORA_ALPHA = 512 # 1024
LORA_DROPOUT = 0.05
# Define LoRA Config
lora_config = LoraConfig(
                 r = LORA_R, # the dimension of the low-rank matrices
                 lora_alpha = LORA_ALPHA, # scaling factor for the weight matrices
                 lora_dropout = LORA_DROPOUT, # dropout probability of the LoRA layers
                 bias="none",
                 task_type="CAUSAL_LM",
                 target_modules=["q_proj", "o_proj", "k_proj"],
)

# Prepare int-8 model for training - utility function that prepares a PyTorch model for int8 quantization training. <https://huggingface.co/docs/peft/task_guides/int8-asr>
model = prepare_model_for_kbit_training(model)
# initialize the model with the LoRA framework
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 50,331,648 || all params: 1,227,096,064 || trainable%: 4.1017


## Setting the Training Arguments


This script fine-tunes a model using the Hugging Face Trainer with settings optimized for training in epochs. It trains the model for `3 epochs` with a learning rate of `1e-4`, uses `fp16` precision for efficiency, and saves the model and configurations to the folder `"olmo"` at the end of each epoch. The Trainer handles training, evaluation, and data preparation automatically.


[optional] if you have a wandb api key, enter it to log the training and Validation loss in wandb dashboard.

In [ ]:
from transformers import TrainingArguments, Trainer
import bitsandbytes
# define the training arguments first.
EPOCHS = 1
LEARNING_RATE = 1e-4
MODEL_SAVE_FOLDER_NAME = "olmo"
training_args = TrainingArguments(
                    output_dir=MODEL_SAVE_FOLDER_NAME,
                    overwrite_output_dir=True,
                    fp16=True, #converts to float precision 16 using bitsandbytes
                    per_device_train_batch_size=1,
                    per_device_eval_batch_size=1,
                    learning_rate=LEARNING_RATE,
                    num_train_epochs=EPOCHS,
                    logging_strategy="epoch",
                    evaluation_strategy="epoch",
                    save_strategy="epoch",
)
# training the model
trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=split_dataset['train'],
        eval_dataset=split_dataset["test"],
        data_collator=data_collator,
)
model.config.use_cache = False
trainer.train()
trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)
trainer.save_model(MODEL_SAVE_FOLDER_NAME)
trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

Epoch,Training Loss,Validation Loss
1,2.352500,2.321052


## Evaluation

Loading the finetuned model.


In [ ]:
eval_model=AutoModelForCausalLM.from_pretrained('./olmo/checkpoint-800',token=os.environ['HF_TOKEN'])
tokenizer=AutoTokenizer.from_pretrained('./olmo/checkpoint-800')

You can observe the change in the modules targeted while finetuning.

In [ ]:
eval_model

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=256, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=256, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
      

Loading a subset of the dataset for testing purposes.

In [ ]:
data=pd.read_csv(s3_url)

dataset = Dataset.from_pandas(data)


test_set=dataset.select(range(6900,7400))

test_set

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 500
})

In [ ]:
tokenizer.pad_token=tokenizer.eos_token
tokenizer.pad_token_id=tokenizer.eos_token_id



`Model and Tokenizer`: Uses the specified eval_model and tokenizer.

`Task: text-generation`, generating new text based on an input prompt.
Generation Settings:

`Maximum number of tokens` in the output: `512`.

Sampling `temperature: 0.3` (low value for more deterministic outputs).

`Top-p` sampling: `1` (includes all tokens for generation).

Enables sampling with `do_sample=True`.

`Repetition penalty`: `1.1` (reduces repetitive text).

The pipeline ensures the generated text includes the full context from the input prompt.

In [ ]:
import transformers

pipeline=transformers.pipeline(
    model=eval_model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    max_new_tokens=20,
    temperature=0.6,
    top_p=1,
    do_sample=False,
    repetition_penalty=1.1
)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
from langchain import HuggingFacePipeline
finetuned_llm= HuggingFacePipeline(pipeline=pipeline)

### Evaluating the response of finetuned model against the actual expected response from the dataset.

In [ ]:
i=test_set[0]
# print(i['question'])
prediction=finetuned_llm.invoke("Predict the sentiment of the review in one word postive or negative." +i['review'])
print(prediction[len(i['review']):])
print('---------------------------------')
print(i['sentiment'])
print('---------------------------------------')
print('---------------------------------------')

leasure, great fun and best to watch it with friends. (10 out of 10)
 sentiment:positive
---------------------------------
positive
---------------------------------------
---------------------------------------
